[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ferdinandre/deepvision/blob/main/milestone1.ipynb)

In [ ]:
!pip install kaggle

In [ ]:
from getpass import getpass
username = getpass('Enter the username: ')

secret = getpass('Enter the secret value: ')

In [3]:
!echo "{\"username\":\"andrferdinnd\",\"key\":\"a4e2c7535dc88232d3e00cb5099b3995\"}"  > ~/.kaggle/kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!touch ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!cd ~/kaggle ; ls -l
!cat ~/.kaggle/kaggle.json

/bin/bash: line 1: cd: /root/kaggle: No such file or directory
total 4
drwxr-xr-x 1 root root 4096 Oct 24 13:20 sample_data
{"username":"andrferdinnd","key":"a4e2c7535dc88232d3e00cb5099b3995"}


In [5]:
!kaggle competitions download -c isic-2024-challenge

 99% 1.99G/2.00G [00:34<00:00, 77.3MB/s]
100% 2.00G/2.00G [00:34<00:00, 61.5MB/s]


In [ ]:
!unzip isic-2024-challenge.zip -d .

In [ ]:
import pandas as pd
import numpy as np
import csv
import os
from PIL import Image
import h5py
import tensorflow as tf
!pip install matplotlib
import matplotlib as plt
import os
from PIL import Image
import random
import cv2
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#Defining file paths
train_metadata_path = "/content/train-metadata.csv"
image_path = "/content/train-image/image/"

#Read training and testing images
train_metadata = pd.read_csv(train_metadata_path, low_memory=False)

#Print
print(train_metadata.head())

In [ ]:
#Get ID and Target from train_metadata
id_target_train = train_metadata[["isic_id","target"]]

print(id_target_train["target"], id_target_train["isic_id"])

In [18]:
# Count rows where target is 1
count_target_1 = (id_target_train["target"] == 1).sum()

# Count rows where target is 0
count_target_0 = (id_target_train["target"] == 0).sum()

print("Number of rows where target is 1:", count_target_1)
print("Number of rows where target is 0:", count_target_0)

Number of rows where target is 1: 393
Number of rows where target is 0: 400666


In [20]:
import glob

image_files = glob.glob(image_path + "*.jpg")

file_dict = {file_name.split: None for file_name in image_files}

with open(train_metadata_path, mode='r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
          file_dict[row[0] + ".jpg"] = row[1]

In [41]:
class LabeledPicture:
  def __init__(self, image, label):
        """
        Initializes the ImageWithLabel instance.

        Args:
            image: The OpenCV image (numpy array) to store.
            label: The integer label associated with the image.
        """
        self.image = image
        self.label = label

In [72]:
# Define the input directory
input_dir = '/content/train-image/image'

# Define the number of augmentations per image
augmentations_per_image = 3

# List to store original and augmented images
sick_images = []

# Define augmentation functions
def augment_image(image):
    # Randomly flip
    if random.choice([True, False]):
        image = cv2.flip(image, 1)

    # Random rotation
    angle = random.randint(-30, 30)
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    image = cv2.warpAffine(image, M, (w, h))

    # Random translation
    tx, ty = random.randint(-10, 10), random.randint(-10, 10)
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    image = cv2.warpAffine(image, M, (w, h))

    # Random brightness adjustment
    if random.choice([True, False]):
        image = cv2.convertScaleAbs(image, alpha=1, beta=random.randint(-30, 30))

    return image

# Loop through each image in the input directory
for key, value in file_dict.items():
  if value == '1':
    image_path = os.path.join(input_dir, key)
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    sick_images.append(LabeledPicture(image, 1))
    for _ in range(augmentations_per_image):
      augmented_image = augment_image(image)
      labeled_image = LabeledPicture(augmented_image, 1)
      sick_images.append(labeled_image)

# Now, augmented_images contains both original and augmented images in memory
print(f"Total sick images (including augmented): {len(sick_images)}")

Total sick images (including augmented): 1572


In [ ]:
sick_images

In [74]:
# Kivalasztani 1572 0s targetu imaget es betolteni a memoriaba
healthy_images = []


counter = 0
for key, value in file_dict.items():
    if value == '0' and counter < 1572:
        image_path = os.path.join(input_dir, key)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (224, 224))
        labeled_image = LabeledPicture(image, 0)
        healthy_images.append(labeled_image)
        counter += 1

print(f"Total healthy images: {len(healthy_images)}")

Total healthy images: 1572


In [75]:
total_images = healthy_images + sick_images

np.random.shuffle(total_images)


In [76]:
# 80-10-10

X = []
Y = []

for imageInstance in total_images:
  print(imageInstance)


In [91]:
X, Y = zip(*[(labelledImage.image, labelledImage.label) for labelledImage in total_images])

X_normalized = [(image / 255.0) for image in X]

print(X_normalized[0])

[[[0.43529412 0.4627451  0.65882353]
  [0.42745098 0.45490196 0.65098039]
  [0.41176471 0.43921569 0.63529412]
  ...
  [0.34509804 0.38823529 0.63137255]
  [0.34901961 0.38823529 0.63529412]
  [0.34901961 0.39215686 0.63529412]]

 [[0.41568627 0.44313725 0.63921569]
  [0.40784314 0.43529412 0.63137255]
  [0.39607843 0.42352941 0.61960784]
  ...
  [0.34901961 0.39215686 0.63529412]
  [0.35294118 0.39607843 0.63921569]
  [0.35294118 0.39607843 0.63921569]]

 [[0.37647059 0.40392157 0.6       ]
  [0.37647059 0.40392157 0.6       ]
  [0.37254902 0.4        0.59607843]
  ...
  [0.36078431 0.40392157 0.64705882]
  [0.36078431 0.40784314 0.64705882]
  [0.36470588 0.40784314 0.65098039]]

 ...

 [[0.36470588 0.4        0.6627451 ]
  [0.35686275 0.39215686 0.65490196]
  [0.34117647 0.37647059 0.63921569]
  ...
  [0.2745098  0.28235294 0.32156863]
  [0.27058824 0.27843137 0.31764706]
  [0.27058824 0.27843137 0.31764706]]

 [[0.35294118 0.38823529 0.65098039]
  [0.34509804 0.38039216 0.64313725]


In [92]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_normalized, Y, test_size=0.2, random_state=1)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)